<img src="https://storage.googleapis.com/bubble-finder/$y7gQWsp6suqPiOpXJWWdHVzOxLywtOreP53BIKhGqtnikgDMT4SaiZ" align=left width="100" height="200">



## Data Analyst Assignment

### Part I


The UK Premier League plays 38 rounds each season. Please create a standings table for the 2021-2022 entire season.

The standings table should include the following fields:

    
**Position** |**TeamName**	|**GamesPlayed** |**Wins** |**Draws** |**Losses** |**Scored:Received** |**GoalsDifference** |**Points**
:----|----|----|----|----|----|----|----|----
1 |Manchester City|38|29|6|3|99:26|73|93
2 |Liverpool FC|38|28|8|2|94:26|68|92




To do so, please scrape the season fixtures using the following website (or use any other source of your preference):
https://www.worldfootball.net/schedule/eng-premier-league-2021-2022-spieltag/1/


###### *The Competition Rules:*
1. Three points are awarded for a win, one point for a draw, and none for a defeat.

2. If any clubs finish with the same number of points, their position is determined by goal difference, then the number of goals scored. If the teams still cannot be separated, they will be awarded the same position in the table.



*  *Please note that the score in parentheses in the URL represents the half time score, not the final score*

In [1]:
import numpy as np
import pyautogui
import pandas as pd
import requests
import time
import folium
import webbrowser
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup


In [2]:
def round_table(n_round):
    
    """
    Summary: This function takes as an argument the number of rounds 
    for which we want to generate a standings table.
    
    My approach taken can be summarized as
    
    1) Look at snapshot of matches in a given round
    
    2) Since we have two columns for teams, I decided to move 'winners' to column on the left('Team 1')
       and move 'losers' to the column on the right('Team 2')
       
    3) As the column position of teams that had a draw is irrelevant, and given how now
       winners and losers are perfectly separated, I can add number of corresponding points/wins/losses/draws for each
       
    4) Concatenate winners + losers + draws into a final standings table for the given round
    
    5) Function returns the table.
        
    
    """

    data = requests.get("https://www.worldfootball.net/schedule/eng-premier-league-2021-2022-spieltag/" +n_round+"/")
    soup = BeautifulSoup(data.text)
    standings = soup.select("table.standard_tabelle")[0]
    links = standings.find_all("a",href=True)


    team_1 = []
    team_2 = []
    score_team_1 = []
    score_team_2 = []

    for i in range(0,len(links)-2,3):

            team_1.append(links[i].get('title'))
            team_2.append(links[i+1].get('title'))
            score_team_1.append(links[i+2].contents[0][:1])  #Final score for the first team is the first character
            score_team_2.append(links[i+2].contents[0][2:3]) #Final score for the second team is the first character after first':'


    df_round = pd.DataFrame({'Team 1' : team_1,
                            'Team 2' : team_2,
                            'Score Team 1' : pd.Series(score_team_1, dtype='float64'),
                            'Score Team 2' : pd.Series(score_team_2, dtype='float64')})


    df_round['Goals Difference'] = df_round['Score Team 1'] - df_round['Score Team 2']

    #Below we swap values where the winner is Team 2, so that all winners are on Team 1 column

    df_round[['Team 1', 'Team 2']] = df_round[['Team 2', 'Team 1']].where(df_round['Goals Difference'] < 0,
                                                          df_round[['Team 1', 'Team 2']].values)

    df_round[['Score Team 1', 'Score Team 2']] = df_round[['Score Team 2', 'Score Team 1']].where(df_round['Goals Difference'] < 0,
                                                          df_round[['Score Team 1', 'Score Team 2']].values)

    df_round.loc[df_round['Goals Difference']<0, "Goals Difference"] = df_round['Goals Difference'] * -1

    #Now all the winners are on column 'Team 1', so we can remove 'Team 2' 

    winners = df_round[df_round['Goals Difference']>0].drop(columns='Team 2')
    winners['Points'] = 3
    winners[['Wins','Draws','Losses']] = [1,0,0]

    #Similarly, all losers are on column 'Team 2', so we can remove 'Team 1'
    #One caveat: 'Goals Difference' was calculated on the perspective of 'Team 1', so we'll multiply it by minus 1
    
    losers = df_round[df_round['Goals Difference']>0].drop(columns='Team 1')
    losers['Goals Difference'] = losers['Goals Difference']*-1
    losers['Points'] = 0
    losers[['Wins','Draws','Losses']] = [0,0,1]
    
    winners.columns = ['TeamName','Scored','Received','Goals Difference','Points','Wins','Draws','Losses']
    losers.columns = ['TeamName','Received','Scored','Goals Difference','Points','Wins','Draws','Losses']

    
    #Moving now into teams that had draws, we melt the dataframe in order to have one team per row
    
    draws = df_round[df_round['Goals Difference']==0]
    draws = pd.melt(draws, id_vars = ['Score Team 1','Score Team 2','Goals Difference'] , value_vars = ['Team 1','Team 2'])
    draws.rename(columns={"Score Team 1" : "Scored",
                          "Score Team 2" : "Received",
                         "value" : "TeamName"},inplace=True)

    draws.drop(columns='variable',inplace=True)

    draws = draws[['TeamName','Scored','Received','Goals Difference']]
    draws['Points'] = 1
    draws[['Wins','Draws','Losses']] = [0,1,0]

    table_round = (pd.concat([winners,losers,draws]))
    table_round[['Scored','Received','Goals Difference']] = table_round[['Scored','Received','Goals Difference']].astype(int)
    table_round.reset_index(inplace=True,drop=True)
    
    return table_round

In [3]:
order_columns = ['Position','GamesPlayed','TeamName','Wins','Draws',
                 'Losses','Scored','Received','Goals Difference','Points']

In [4]:
final_df = round_table(str(1))

In [5]:
total_rounds = 38 #There are 38 rounds in total for the Premier League

for i in range (2,total_rounds+1,1):
    
    temp_df = round_table(str(i))

    temp_df.set_index("TeamName",inplace=True)
    final_df.set_index("TeamName",inplace=True)

    for ele in ['Scored','Received','Goals Difference','Points','Wins','Draws','Losses']:

        final_df[ele] = final_df[ele] + temp_df[ele]


    final_df.sort_values(by=['Points','Goals Difference','Scored'],ascending=False,inplace=True)
    final_df.reset_index(inplace=True)
    final_df['Position']  = range(1,21,1)
    final_df['GamesPlayed'] = i
    final_df = final_df.reindex(columns=order_columns)

In [6]:
#This is for the edge cases when teams end up with the same number of points, so we apply the criteria specified.

duplicates = final_df[final_df.duplicated(['Points','Goals Difference','Scored'],keep='first')]
duplicates['Position'] = duplicates['Position'] - 1

final_df.loc[duplicates.index] = duplicates

In [7]:
#Coalescing Scored and Received columns into one column

final_df[['Scored','Received']] = final_df[['Scored','Received']].astype(str)
new_col = final_df['Scored'].str.cat(final_df['Received'],sep=':')
final_df.insert(6,"Scored:Received",new_col) #Inserting the new column at the right column index position

In [8]:
#Some styling additions to our final table
       
final_df = final_df.style.set_properties(**{'background-color': 'black',
                           'color': 'white'})

final_df.apply(lambda x: ['background: mediumseagreen' if x.name in [0,1,2,3] 
                              else 'background: mediumslateblue ' if x.name in [4,5,6]
                              else 'background: salmon' if x.name in [17,18,19]
                              else '' for i in x], axis=1)


,Position,GamesPlayed,TeamName,Wins,Draws,Losses,Scored:Received,Scored,Received,Goals Difference,Points
0,1,38,Manchester City,29,6,3,99:26,99,26,73,93
1,2,38,Liverpool FC,28,8,2,94:26,94,26,68,92
2,3,38,Chelsea FC,21,11,6,76:33,76,33,43,74
3,4,38,Tottenham Hotspur,22,5,11,69:40,69,40,29,71
4,5,38,Arsenal FC,22,3,13,61:48,61,48,13,69
5,6,38,Manchester United,16,10,12,57:57,57,57,0,58
6,7,38,West Ham United,16,8,14,60:51,60,51,9,56
7,8,38,Leicester City,14,10,14,62:59,62,59,3,52
8,9,38,Brighton & Hove Albion,12,15,11,42:44,42,44,-2,51
9,10,38,Wolverhampton Wanderers,15,6,17,38:43,38,43,-5,51


<span style="color:mediumseagreen">||||||||||||||||||||||||||||||||</span> - Champions League <br>
<span style="color:mediumslateblue">||||||||||||||||||||||||||||||||</span> - The league no one cares about ("Europa League") <br>
<span style="color:salmon">||||||||||||||||||||||||||||||||</span> - Relegation <br>

### Part II

Based on the output you have created in Part I, please add the name of the home stadium of each team in the Premier League



In [9]:
#This is a scraper to get the stadiums for each team plus their coordinates

stadiums = []
latitudes = []
longitudes = []

op = webdriver.ChromeOptions()
op.add_argument('headless')
chromedrive_path = './chromedriver' 
#driver = webdriver.Chrome(chromedrive_path,options=op)
driver = webdriver.Chrome(chromedrive_path)


for team in final_df.data['TeamName']:

    driver.get("https://google.com/maps/search/"+str(team) +" stadium")
    time.sleep(5)

    #print(driver.title)
    stadiums.append(driver.title)
    latitudes.append(driver.current_url.split("!3d")[1].split("!4d")[0])
    longitudes.append(driver.current_url.split("!3d")[1].split("!4d")[1])
    time.sleep(3)

driver.quit()

/var/folders/v7/03vsm6bj4ksgbclgtwx02_2c0000gn/T/ipykernel_730/2981726800.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path)


In [10]:
#Removing 'Google Maps' substring from every element in our list

stadiums = [s.replace(' - Google Maps', '') for s in stadiums]

In [11]:
#Create the Stadium column based on the list

final_df.data['Stadium'] = stadiums

In [12]:
final_df

,Position,GamesPlayed,TeamName,Wins,Draws,Losses,Scored:Received,Scored,Received,Goals Difference,Points,Stadium
0,1,38,Manchester City,29,6,3,99:26,99,26,73,93,Etihad Stadium
1,2,38,Liverpool FC,28,8,2,94:26,94,26,68,92,Anfield
2,3,38,Chelsea FC,21,11,6,76:33,76,33,43,74,Stamford Bridge
3,4,38,Tottenham Hotspur,22,5,11,69:40,69,40,29,71,Tottenham Hotspur Stadium
4,5,38,Arsenal FC,22,3,13,61:48,61,48,13,69,Emirates Stadium
5,6,38,Manchester United,16,10,12,57:57,57,57,0,58,Old Trafford
6,7,38,West Ham United,16,8,14,60:51,60,51,9,56,London Stadium
7,8,38,Leicester City,14,10,14,62:59,62,59,3,52,King Power Stadium
8,9,38,Brighton & Hove Albion,12,15,11,42:44,42,44,-2,51,American Express Community Stadium
9,10,38,Wolverhampton Wanderers,15,6,17,38:43,38,43,-5,51,Molineux Stadium


### Part III

Please visualize on a map the locations of all the teams' stadiums and show relevant data in the tooltip.

In [13]:
#Create a dataframe with coordinates for every stadium

stadium_location_df = pd.DataFrame({'TeamName' : final_df.data['TeamName'],
                                    'Position' : final_df.data['Position'],
                                    'Points' : final_df.data['Points'],  
                                    'Goals Scored/Received' : final_df.data['Scored:Received'],
                        'Stadium' : stadiums,
                        'Latitude' : pd.Series(latitudes, dtype='float64'),
                        'Longitude' : pd.Series(longitudes, dtype='float64')})


In [14]:
#Creating a dictionary for icons from web

dic_icons = {"Arsenal FC" : "https://icons.iconarchive.com/icons/giannis-zographos/english-football-club/256/Arsenal-FC-icon.png",
            "Aston Villa" : "https://icons.iconarchive.com/icons/giannis-zographos/english-football-club/256/Aston-Villa-icon.png",
             "Brentford FC" : "https://seeklogo.com/images/B/brentford-fc-logo-E928FFBD93-seeklogo.com.png",
             "Brighton & Hove Albion" : "https://upload.wikimedia.org/wikipedia/sco/thumb/f/fd/Brighton_%26_Hove_Albion_logo.svg/1200px-Brighton_%26_Hove_Albion_logo.svg.png",
             "Burnley FC" : "https://upload.wikimedia.org/wikipedia/en/thumb/6/62/Burnley_F.C._Logo.svg/1200px-Burnley_F.C._Logo.svg.png",
             "Chelsea FC" : "https://icons.iconarchive.com/icons/giannis-zographos/english-football-club/256/Chelsea-FC-icon.png",
             "Crystal Palace" : "https://upload.wikimedia.org/wikipedia/en/thumb/a/a2/Crystal_Palace_FC_logo_%282022%29.svg/1200px-Crystal_Palace_FC_logo_%282022%29.svg.png",
             "Everton FC" : "https://upload.wikimedia.org/wikipedia/sco/thumb/7/7c/Everton_FC_logo.svg/1200px-Everton_FC_logo.svg.png",
             "Leeds United" : "https://icons.iconarchive.com/icons/giannis-zographos/english-football-club/256/Leicester-City-icon.png",
             "Liverpool FC" : "https://icons.iconarchive.com/icons/giannis-zographos/english-football-club/256/Liverpool-FC-icon.png",
             "Manchester City" : "https://cdn-icons-png.flaticon.com/512/824/824726.png",
             "Leicester City" : "https://icons.iconarchive.com/icons/giannis-zographos/english-football-club/256/Leicester-City-icon.png",
             "Manchester United" : "https://cdn-icons-png.flaticon.com/512/824/824727.png",
             "Newcastle United" : "https://icons.iconarchive.com/icons/giannis-zographos/english-football-club/256/Newcastle-United-icon.png",
             "Norwich City" : "https://icons.iconarchive.com/icons/giannis-zographos/british-football-club/256/Norwich-City-icon.png",
             "Southampton FC" : "https://icons.iconarchive.com/icons/giannis-zographos/english-football-club/256/Southampton-FC-icon.png",
             "Tottenham Hotspur" : "https://upload.wikimedia.org/wikipedia/en/thumb/b/b4/Tottenham_Hotspur.svg/1200px-Tottenham_Hotspur.svg.png",
             "Watford FC" : "https://icons.iconarchive.com/icons/giannis-zographos/english-football-club/256/Watford-FC-icon.png",
             "West Ham United" : "https://upload.wikimedia.org/wikipedia/en/thumb/c/c2/West_Ham_United_FC_logo.svg/1200px-West_Ham_United_FC_logo.svg.png",
             "Wolverhampton Wanderers" : "https://upload.wikimedia.org/wikipedia/sco/thumb/f/fc/Wolverhampton_Wanderers.svg/1200px-Wolverhampton_Wanderers.svg.png"
            }

In [15]:
stadium_location_df['Icon link'] = stadium_location_df['TeamName']
stadium_location_df.replace({"Icon link": dic_icons},inplace=True)

In [16]:
stadium_location_df['Icon link'] = stadium_location_df['Icon link'].to_list()

In [17]:
#Scraping wikipedia articles for each team

wikipedia_links = []

for team in stadium_location_df['TeamName']:
    
    if team == "Crystal Palace":
        
        team = "Crystal Palace F.C."

    op = webdriver.ChromeOptions()
    op.add_argument('headless')
    chromedrive_path = './chromedriver' 
    driver = webdriver.Chrome(chromedrive_path,options=op)
    

    driver.get("https://en.wikipedia.org/wiki/Main_Page")

    search_bar = driver.find_element("xpath",'//*[@id="searchInput"]')


    search_bar.send_keys(str(team))
    
    time.sleep(2)

    search_bar.send_keys(Keys.ENTER)

    wikipedia_links.append(driver.current_url)

driver.quit()

/var/folders/v7/03vsm6bj4ksgbclgtwx02_2c0000gn/T/ipykernel_730/620669573.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path,options=op)


In [18]:
#Youtube videos containing goals of season for every team


youtube_links = []

op = webdriver.ChromeOptions()
op.add_argument('headless')
chromedrive_path = './chromedriver' 
driver = webdriver.Chrome(chromedrive_path,options=op)


for team in stadium_location_df['TeamName']:

    driver.get("https://www.youtube.com/")
    time.sleep(1)
    search_bar = driver.find_element("name","search_query")
    

    search_bar.send_keys('all ' + str(team) + ' goals 2021/22')

    time.sleep(3)

    search_bar.send_keys(Keys.ENTER)
    

    search_bar = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="video-title"]/yt-formatted-string')))


    actionchains = ActionChains(driver)
    actionchains.context_click(search_bar).perform()

    pyautogui.press('down', presses=1)
    pyautogui.press('enter')

    time.sleep(3)
    driver.switch_to.window(driver.window_handles[-1])
    youtube_links.append(driver.current_url)
    
    driver.close()

    driver.switch_to.window(driver.window_handles[0])
    
    
driver.quit()

/var/folders/v7/03vsm6bj4ksgbclgtwx02_2c0000gn/T/ipykernel_730/1879971310.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path)


In [19]:
#Youtube videos containing goals of season for every team

youtube_stadium_links = []

op = webdriver.ChromeOptions()
op.add_argument('headless')
chromedrive_path = './chromedriver' 
driver = webdriver.Chrome(chromedrive_path,options=op)


for stadium in stadium_location_df['Stadium']:

    driver.get("https://www.youtube.com/")
    time.sleep(1)
    search_bar = driver.find_element("name","search_query")

    search_bar.send_keys(str(stadium) + ' tour')

    time.sleep(3)

    search_bar.send_keys(Keys.ENTER)

    search_bar = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="video-title"]/yt-formatted-string')))


    actionchains = ActionChains(driver)
    actionchains.context_click(search_bar).perform()

    pyautogui.press('down', presses=1)
    pyautogui.press('enter')

    time.sleep(3)
    driver.switch_to.window(driver.window_handles[-1])
    youtube_stadium_links.append(driver.current_url)
    
    driver.close()

    driver.switch_to.window(driver.window_handles[0])
    
    
    #print(youtube_links[-1])
    
driver.quit()

/var/folders/v7/03vsm6bj4ksgbclgtwx02_2c0000gn/T/ipykernel_730/2427065404.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedrive_path)


In [20]:
stadium_location_df['Wikipedia link'] = wikipedia_links
stadium_location_df['Youtube link'] = youtube_links
stadium_location_df['Stadium Youtube link'] = youtube_stadium_links

In [21]:
my_map = folium.Map(location=[stadium_location_df.Latitude.mean(), stadium_location_df.Longitude.mean()], zoom_start=7, control_scale=True)

In [22]:
for index, location_info in stadium_location_df.iterrows():
    icon = folium.features.CustomIcon(stadium_location_df['Icon link'][index],icon_size=(28, 30))
    folium.Marker([location_info["Latitude"], location_info["Longitude"]],
                  tooltip= "<b>Team: "+stadium_location_df['TeamName'][index] +  "<br><b>Stadium: <b>" + location_info["Stadium"] + "<br><b>Final Position 2021/2022: <b>" 
                  + stadium_location_df["Position"][index].astype(str) + "<br><b>Points: <b>"+ stadium_location_df["Points"][index].astype(str) + "<br><b>Goals Scored/Received: <b>"+ stadium_location_df["Goals Scored/Received"][index],
                  popup= '<h1>'+stadium_location_df['TeamName'][index]+'</h1><br><b>Team&nbsp;history:&nbsp;<b><a href='+stadium_location_df['Wikipedia link'][index]
                  +' target = "_blank">link </a><br>Highlights&nbsp;2021-2022:&nbsp;<a href='+stadium_location_df['Youtube link'][index]+' target = "_blank">link </a>'
                  +'<br>Take&nbsp;a tour:&nbsp;<a href='+stadium_location_df['Stadium Youtube link'][index]+' target = "_blank">link </a>',icon=icon,
                 ).add_to(my_map)

my_map.save('map.html')    


webbrowser.open('map.html', new=2)

True

# Click on map icons for more info

In [23]:
my_map



Be creative    





##### Good Luck
